In [1]:
import sys
from pathlib import Path

In [5]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [9]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint

In [36]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("")
    print(str(query))
    results = session.exec(query).fetchall()
    pprint(results)

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT :param_1
[EventModel(description='', time=datetime.datetime(2025, 3, 21, 17, 0, 14, 793459, tzinfo=datetime.timezone.utc), id=1, updated_at=datetime.datetime(2025, 3, 21, 17, 0, 14, 793466, tzinfo=datetime.timezone.utc), page='pricing'),
 EventModel(description='', time=datetime.datetime(2025, 3, 21, 17, 0, 14, 808389, tzinfo=datetime.timezone.utc), id=2, updated_at=datetime.datetime(2025, 3, 21, 17, 0, 14, 808394, tzinfo=datetime.timezone.utc), page='/about'),
 EventModel(description='', time=datetime.datetime(2025, 3, 21, 17, 0, 14, 813534, tzinfo=datetime.timezone.utc), id=3, updated_at=datetime.datetime(2025, 3, 21, 17, 0, 14, 813538, tzinfo=datetime.timezone.ut

In [ ]:
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    pages = ['/about', '/contact', '/pages', '/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query = (
        select(
            bucket,
            EventModel.page,
            func.count()
        )
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(
            bucket,
            EventModel.page,
        )
        .order_by(
            bucket,
            EventModel.page,
        )
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    # print(compiled_query)
    results = session.exec(query).fetchall()
    pprint(results)

SELECT time_bucket('1 day'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel 
WHERE eventmodel.time > '2025-03-21 17:12:27.563396+00:00' AND eventmodel.time <= '2025-03-21 19:12:27.563402+00:00' AND eventmodel.page IN ('/about', '/contact', '/pages', '/pricing') GROUP BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page
